In [21]:
import pandas as pd

In [22]:
df_accounts = pd.read_csv('data/accounts.csv')
df_alliance_ff = pd.read_csv('data/alliance_ff.csv')
df_alliance_membership = pd.read_csv('data/alliance_membership.csv')
df_chat_messages_1 = pd.read_csv('data/chat_messages_1.csv', dtype='unicode')
df_chat_messages_2 = pd.read_csv('data/chat_messages_2.csv', dtype='unicode')

Clean dataset chat_messages_1   

Drop unnecessary columns

In [23]:
columns_to_drop = ['risk', 'filter_detected_language', 'is_family_friendly', 'BULLYING', 'VIOLENCE', 'RELATIONSHIP_SEXUAL_CONTENT', 'VULGARITY', 'DRUGS_ALCOHOL', 'IN_APP', 'ALARM', 'FRAUD', 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS']
df_chat_messages_1.drop(columns_to_drop, inplace=True, axis=1)
df_chat_messages_1.to_csv("modified_chat_messages_1.csv", index=False)

Clean dataset chat_messages_2

Drop unnecessary columns

In [ ]:
columns_to_drop2 = ['risk', 'filter_detected_language', 'is_family_friendly', 'BULLYING', 'VIOLENCE', 'RELATIONSHIP_SEXUAL_CONTENT', 'VULGARITY', 'DRUGS_ALCOHOL', 'IN_APP', 'ALARM', 'FRAUD', 'HATE_SPEECH', 'RELIGIOUS', 'WEBSITE', 'CHILD_GROOMING', 'PUBLIC_THREAT', 'EXTREMISM', 'SUBVERSIVE', 'SENTIMENT', 'POLITICS']
df_chat_messages_2.drop(columns_to_drop2, inplace=True, axis=1)
df_chat_messages_2.to_csv("modified_chat_messages_2.csv", index=False)

Load Model

In [27]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import timm
model_reloaded = timm.create_model('hf_hub:nateraw/resnet18-random', pretrained=True)